In [ ]:
import pandas as pd
import numpy as np

def generate_random_object():
    return {
        'x': np.random.uniform(-10, 10),
        'y': np.random.uniform(-10, 10),
        'z': np.random.uniform(0, 10),  # высота от 0 до 10
        'size': np.random.uniform(10, 15),
        'figure': np.random.choice(['circle', 'rectangle']),
        'rotation': np.random.uniform(0, 360),  # поворот в градусах в плоскости XY
        'tilt': np.random.uniform(-45, 45)  # наклон по оси Z в градусах
    }

def generate_data(num_str=100):
    df = pd.DataFrame(columns=['x1', 'y1', 'z1', 'size1', 'figure1', 'rotation1', 'tilt1',
                               'x2', 'y2', 'z2', 'size2', 'figure2', 'rotation2', 'tilt2', 'collision'])
    
    for i in range(num_str):
        obj1 = generate_random_object()
        obj2 = generate_random_object()
        
        temp_df = pd.DataFrame([{
            'x1': obj1['x'], 'y1': obj1['y'], 'z1': obj1['z'], 'size1': obj1['size'], 
            'figure1': obj1['figure'], 'rotation1': obj1['rotation'], 'tilt1': obj1['tilt'],
            'x2': obj2['x'], 'y2': obj2['y'], 'z2': obj2['z'], 'size2': obj2['size'], 
            'figure2': obj2['figure'], 'rotation2': obj2['rotation'], 'tilt2': obj2['tilt'],
            'collision': collision(obj1, obj2)
        }])
        
        df = pd.concat([df, temp_df], ignore_index=True)
    
    return df

def distance(x1, x2, y1, y2, z1, z2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

def get_circle_radius(size):
    return size / 2

def get_square_half_side(size):
    return size / 2

def check_circle_square_collision(cx, cy, cz, radius, rx, ry, rz, size, rotation, tilt):
    # Упрощенная проверка: проекция круга на плоскость прямоугольника
    # с учетом поворота прямоугольника и наклона
    half_side = get_square_half_side(size)
    
    # Учет поворота прямоугольника в плоскости XY
    dx = cx - rx
    dy = cy - ry
    dz = cz - rz
    
    # Поворот координат круга относительно прямоугольника
    rot_rad = np.radians(rotation)
    x_rot = dx * np.cos(rot_rad) + dy * np.sin(rot_rad)
    y_rot = -dx * np.sin(rot_rad) + dy * np.cos(rot_rad)
    
    # Проверка по оси Z с учетом наклона (tilt)
    tilt_rad = np.radians(tilt)
    z_proj = dz * np.cos(tilt_rad)
    
    # Проверка столкновения в проекции
    closest_x = max(-half_side, min(x_rot, half_side))
    closest_y = max(-half_side, min(y_rot, half_side))
    dist_xy = np.sqrt((x_rot - closest_x)**2 + (y_rot - closest_y)**2)
    
    return dist_xy < radius and abs(z_proj) < radius + half_side

def get_rectangle_vertices(x, y, z, size, rotation, tilt):
    """Вычисляет вершины прямоугольника в 3D с учетом поворота и наклона."""
    half_size = size / 2
    # Локальные координаты вершин в плоскости XY
    vertices = [
        [-half_size, -half_size, 0],
        [half_size, -half_size, 0],
        [half_size, half_size, 0],
        [-half_size, half_size, 0]
    ]
    
    # Поворот в плоскости XY
    rot_rad = np.radians(rotation)
    cos_r = np.cos(rot_rad)
    sin_r = np.sin(rot_rad)
    
    # Наклон по оси Z (вращение вокруг оси X)
    tilt_rad = np.radians(tilt)
    cos_t = np.cos(tilt_rad)
    sin_t = np.sin(tilt_rad)
    
    rotated_vertices = []
    for vx, vy, vz in vertices:
        # Поворот в XY
        x_rot = vx * cos_r - vy * sin_r
        y_rot = vx * sin_r + vy * cos_r
        z_rot = vz
        
        # Наклон по Z (вращение вокруг X)
        y_tilt = y_rot * cos_t - z_rot * sin_t
        z_tilt = y_rot * sin_t + z_rot * cos_t
        
        # Смещение к центру объекта
        rotated_vertices.append([x + x_rot, y + y_tilt, z + z_tilt])
    
    return np.array(rotated_vertices)

def project_vertices(vertices, axis):
    """Проецирует вершины на ось и возвращает минимальную и максимальную проекции."""
    dots = np.dot(vertices, axis)
    return np.min(dots), np.max(dots)

def get_axes(rect1_vertices, rect2_vertices):
    """Возвращает оси для проверки SAT (нормали к граням)."""
    axes = []
    
    # Грани первого прямоугольника
    for i in range(4):
        v1 = rect1_vertices[i]
        v2 = rect1_vertices[(i + 1) % 4]
        edge = v2 - v1
        # Нормаль к грани в 3D
        normal = np.array([-edge[1], edge[0], 0])  # Перпендикуляр в плоскости XY
        normal = normal / np.linalg.norm(normal) if np.linalg.norm(normal) > 0 else normal
        axes.append(normal)
        
        # Учет наклона: добавляем нормаль с учетом Z
        normal_z = np.cross(edge, [0, 0, 1])  # Векторное произведение с осью Z
        normal_z = normal_z / np.linalg.norm(normal_z) if np.linalg.norm(normal_z) > 0 else normal_z
        axes.append(normal_z)
    
    # Грани второго прямоугольника
    for i in range(4):
        v1 = rect2_vertices[i]
        v2 = rect2_vertices[(i + 1) % 4]
        edge = v2 - v1
        normal = np.array([-edge[1], edge[0], 0])
        normal = normal / np.linalg.norm(normal) if np.linalg.norm(normal) > 0 else normal
        axes.append(normal)
        
        normal_z = np.cross(edge, [0, 0, 1])
        normal_z = normal_z / np.linalg.norm(normal_z) if np.linalg.norm(normal_z) > 0 else normal_z
        axes.append(normal_z)
    
    return axes

def rectangles_collide(obj1, obj2):
    """Проверяет столкновение двух прямоугольников с учетом поворота и наклона."""
    # Получаем вершины обоих прямоугольников
    rect1_vertices = get_rectangle_vertices(obj1['x'], obj1['y'], obj1['z'], 
                                            obj1['size'], obj1['rotation'], obj1['tilt'])
    rect2_vertices = get_rectangle_vertices(obj2['x'], obj2['y'], obj2['z'], 
                                            obj2['size'], obj2['rotation'], obj2['tilt'])
    
    # Получаем оси для проверки
    axes = get_axes(rect1_vertices, rect2_vertices)
    
    # Проверяем проекции на каждую ось
    for axis in axes:
        proj1_min, proj1_max = project_vertices(rect1_vertices, axis)
        proj2_min, proj2_max = project_vertices(rect2_vertices, axis)
        
        # Если есть зазор между проекциями, столкновения нет
        if proj1_max < proj2_min or proj2_max < proj1_min:
            return False
    
    # Если на всех осях есть пересечение, столкновение есть
    return True

def collision(obj1, obj2):
    # Проверка по оси Z для всех случаев
    z_dist = abs(obj1['z'] - obj2['z'])
    max_z_overlap = get_circle_radius(obj1['size']) + get_circle_radius(obj2['size'])  # максимальная высота для пересечения
    
    if z_dist > max_z_overlap:
        return False  # нет столкновения по Z

    # Два круга
    if obj1['figure'] == 'circle' and obj2['figure'] == 'circle':
        dist = distance(obj1['x'], obj2['x'], obj1['y'], obj2['y'], obj1['z'], obj2['z'])
        return dist < get_circle_radius(obj1['size']) + get_circle_radius(obj2['size'])

    # Два прямоугольника
    elif obj1['figure'] == 'rectangle' and obj2['figure'] == 'rectangle':
        # Упрощение: проверяем AABB (Axis-Aligned Bounding Box) с учетом поворота
        if obj1['figure'] == 'rectangle' and obj2['figure'] == 'rectangle':
            return rectangles_collide(obj1, obj2)
    # Здесь можно добавить проверки для других случаев (круг-круг, круг-прямоугольник)

    # Круг и прямоугольник
    elif {obj1['figure'], obj2['figure']} == {'circle', 'rectangle'}:
        circle_obj = obj1 if obj1['figure'] == 'circle' else obj2
        rect_obj = obj2 if obj1['figure'] == 'circle' else obj1
        
        return check_circle_square_collision(
            circle_obj['x'], circle_obj['y'], circle_obj['z'], get_circle_radius(circle_obj['size']),
            rect_obj['x'], rect_obj['y'], rect_obj['z'], rect_obj['size'],
            rect_obj['rotation'], rect_obj['tilt']
        )

# Пример использования
df = generate_data(50)
df

C:\Users\user\AppData\Local\Temp\ipykernel_15676\744448986.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


,x1,y1,z1,size1,figure1,rotation1,tilt1,x2,y2,z2,size2,figure2,rotation2,tilt2,collision
0,1.054257,7.396838,4.249350,13.246936,circle,60.578102,-33.453543,-5.756252,7.623078,4.712530,13.715260,rectangle,99.113929,21.489706,True
1,3.131412,4.061867,8.342537,11.455587,circle,178.841608,7.032312,-9.648856,1.847286,8.411080,13.895841,rectangle,276.797922,36.454295,False
2,-1.795198,-7.463986,3.678342,13.955822,rectangle,152.893733,24.915502,-0.555671,7.600060,3.365436,14.994284,circle,286.043558,3.269051,True
3,-0.512427,6.732441,2.668155,13.657109,rectangle,348.495282,-38.477862,-6.304838,0.360621,2.009011,11.928619,rectangle,14.367894,35.541944,True
4,1.588387,9.857570,4.456181,11.669804,rectangle,318.568042,-2.812820,0.239666,1.531241,3.774254,14.149824,rectangle,335.798231,16.821201,True
5,-2.813520,2.672290,3.230775,10.236146,rectangle,314.042916,-35.396323,-3.233297,-9.593784,1.329339,13.267097,rectangle,16.115193,-27.061434,False
6,-7.813858,-5.396291,4.681947,14.367262,rectangle,131.967655,36.086493,3.536548,0.185490,1.129683,11.894997,rectangle,179.175144,-6.325632,True
7,0.093855,-8.038978,0.441738,10.841764,circle,160.961491,3.175462,2.882515,4.788146,0.058769,10.614889,rectangle,151.481774,31.418115,False
8,-0.820022,-0.485289,3.713869,14.680150,rectangle,104.745768,43.385292,-4.615250,8.304630,2.106278,10.300101,rectangle,21.616833,4.438160,True
9,6.235882,2.145043,1.447890,11.155074,circle,259.330940,-43.814854,0.008611,2.504863,2.226086,11.989559,rectangle,242.930367,6.485935,True
